В данном блокноте рассматривается пример использования диаграммы влияния для выбора оптимальной стратегии при решении задачи с неопределенностью.

Блокнот является своего рода пояснением к [слайдам](https://avponomarev.bitbucket.io/slides/Курсовая_вводная_БС.pdf) и в нем рассматривается тот же "игрушечный" пример из области медицинской диагностики. А именно:

Вероятность появление некоторого заболевания составляет $0.01$. Заболевание может проявляться одним из двух симптомов (С1 и С2). Вероятность проявления симптома С1 при наличии и отсутствии заболевания составляет $0.7$ и $0.01$ соответственно, а вероятность проявления симптома С2 при наличии и отсутствии заболевания составляет $0.9$ и $0.3$ соответственно. Предполагается, что для данного заболевания существует всего один вид терапии. Известны также значения функции полезности, отражающей субъективные неудобства от каждого возможного развития событий:

| Болезнь | Лечить?  |  Эффект  |
|---------|----------|----------|
|  Да     |  Да      |   -1     |
|  Да     |  Нет     |   -40    |
|  Нет    |  Да      |   -10    |
|  Нет    |  Нет     |    0     |

Для описания данной задачи воспользуемся возможностями библиотеки pyAgrum. Построим модель (байесовскую сеть), а потом ответим с ее помощью на несколько вопросов, касающихся выбора оптимальной стратегии для различных ситуаций.

Установить библиотеку можно с помощью [следующих команд](https://agrum.gitlab.io/pages/pyagrum-installation.html):

    conda install -c conda-forge pyagrum

или 

    pip install pyagrum


Примечание! Вообще-то pyAgrum поддерживает возможность создания диаграмм влияния, но в текущих версиях вывод на них работает не совсем корректно. Учитывая, что диаграммы влияния являются расширением байесовских сетей, в данном блокноте мы будем использовать байесовские сети для вычисления вероятностей сложных событий, а часть, связанную с определением (ожидаемых) полезностей и оптимальных управлений будем осуществлять вручную.


In [1]:
import os

%matplotlib inline
from pylab import *
import matplotlib.pyplot as plt

In [2]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

# Создание байесовской сети

Для того, чтобы полностью специфицировать сеть, необходимо задать:

- переменные (вершины трех типов - они же вершины);
- связи между переменными (дуги);
- таблицы условных вероятностей.

В библиотеке pyAgrum для описания диаграммы влияния предназначен класс `BayesNet`:

In [3]:
medicine_bn = gum.BayesNet()
print(medicine_bn)

BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0}


Очевидно, "свежесозданная" сеть пуста - в ней нет ни узлов, ни дуг. Создадим узлы и дуги в соответствии с описанием предметной области, приведенным выше.

## Переменные (и вероятностные узлы)

pyAgrum поддерживает несколько видов переменных. Для данной задачи проще всего использовать, так называемые, `LabelizedVariable`, то есть переменные, которые могут принимать одно из фиксированного набора значений, каждое из которых имеет свою метку. Например, переменная Disease (Болезнь) может принимать одно из двух значений 'Нет' и 'Да'.


In [4]:
# Создадим переменную 
# Первый параметр - имя переменной, второй - опциональный комментарий, а третий - количство значений
va = gum.LabelizedVariable('Disease', 'Is the patient sick?', 2)
va

(gum::LabelizedVariable@0x23adf0aa0e0) Disease<0,1>

Пока что метки (возможные значения переменной Disease) - 0 и 1. С учетом того, что 0 легко можно трактовать как отсутствие чего-то, а 1 - как присутствие, можно оставить описание переменной и в таком виде, однако изменим метки на более осмысленные:

In [5]:
va.changeLabel(0, 'No')
va.changeLabel(1, 'Yes')
va

(gum::LabelizedVariable@0x23adf0aa0e0) Disease<No,Yes>

Наконец, добавим переменную в модель:

In [6]:
medicine_bn.add(va)

0

Аналогично создадим и другие переменные:

In [7]:
# Симптом 1
medicine_bn.add(gum.LabelizedVariable('Symptom1', 'Does the patient have the symptom 1?', 2))
# Симптом 2
medicine_bn.add(gum.LabelizedVariable('Symptom2', 'Does the patient have the symptom 2?', 2))

2

## Дуги

Необходимо связать узлы, соответствующие созданным переменным в струкуру, отражающую их взаимосвязь:

In [8]:
# Наличие болезни влияет на проявление симптомов и на общее самочувствие пациента:
medicine_bn.addArc(medicine_bn.idFromName('Disease'), medicine_bn.idFromName('Symptom1'))
medicine_bn.addArc(medicine_bn.idFromName('Disease'), medicine_bn.idFromName('Symptom2'))

In [9]:
medicine_bn

(gum::BayesNet<double>@0x23adf0a97f0) BN{nodes: 3, arcs: 2, domainSize: 8, dim: 10}

## Таблицы условных вероятностей

Наконец, осталось задать условные вероятности значений переменных. Доступ к таблице условных вероятностей осуществляется с помощью метода `cpt` модели, в качестве параметра которому нужно передать идентификатор узла:

In [10]:
medicine_bn.cpt(medicine_bn.idFromName('Disease'))

(gum::Potential<double>@0x23adf0aa060) <Disease:No> :: 0 /<Disease:Yes> :: 0

Для установки значений условных вероятностей можно воспользоваться методом `fillWith` таблицы, передав в него последовательность вероятностей (по значениям переменной):

In [11]:
medicine_bn.cpt(medicine_bn.idFromName('Disease')).fillWith([0.99, 0.01])
medicine_bn.cpt(medicine_bn.idFromName('Disease'))

(gum::Potential<double>@0x23adf0aa060) <Disease:No> :: 0.99 /<Disease:Yes> :: 0.01

Несколько более сложный случай - задание условных вероятностей для вершин, у которых есть входящие дуги (таблицы таких вершин, очевидно, включают все возможные значения переменных, от которых они зависят). Для правильной установки нужно понимать порядок, в котором упомянуты переменные в таблице:

In [12]:
medicine_bn.cpt(medicine_bn.idFromName('Symptom1')).var_names

['Disease', 'Symptom1']

Соответственно, таблица выглядит так (переменная Disease - первая):

| Disease | Symptom1 | p    |
|---------|----------|------|
|  No     |    0     | 0.99 |
|  No     |    1     | 0.01 |
|  Yes    |    0     | 0.3  |
|  Yes    |    1     | 0.7  |


In [13]:
medicine_bn.cpt(medicine_bn.idFromName('Symptom1')).fillWith([0.99, 0.01, 0.3, 0.7])

(gum::Potential<double>@0x23adf0ad6c0) <Symptom1:0|Disease:No> :: 0.99 /<Symptom1:1|Disease:No> :: 0.01 /<Symptom1:0|Disease:Yes> :: 0.3 /<Symptom1:1|Disease:Yes> :: 0.7

Это же можно делать и с помощью словарей (пожалуй, наиболее читаемый способ):

In [14]:
symptom2 = medicine_bn.idFromName('Symptom2')

medicine_bn.cpt(symptom2)[{'Disease': 'No'}] = [0.7, 0.3]   # два значения: для Symptom2=0 и Symptom2=1 соответственно
medicine_bn.cpt(symptom2)[{'Disease': 'Yes'}] = [0.1, 0.9]

medicine_bn.cpt(symptom2)

(gum::Potential<double>@0x23ae59e0e60) <Symptom2:0|Disease:No> :: 0.7 /<Symptom2:1|Disease:No> :: 0.3 /<Symptom2:0|Disease:Yes> :: 0.1 /<Symptom2:1|Disease:Yes> :: 0.9

На этом спецификация сети завершена. Обратите внимание, что мы нигде не задали полезности исходов и "независимую" переменную, соответсвующую принимаемому решению (лечить или не лечить). Их просто нельзя представить с помощью формализма байесовских сетей (в отличие от диаграмм влияния). 

# Запросы

Построенная модель может быть использована для ответа на различные вопросы.

Например, можно оценить вероятность заболевания при наличии тех или иных симптомов:

In [15]:
# Создадим экземпляр "машины вывода".
# Здесь VariableElimination - это один из распространенных алгоритмов
# точного вывода на БС
ie=gum.VariableElimination(medicine_bn)

# Рассчет всех апостериорных вероятностей
ie.makeInference()

# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(gum::Potential<double>@0x23ae59e0a00) <Disease:No> :: 0.99 /<Disease:Yes> :: 0.01

Очевидно, в данном случае апостериорная вероятность совпадает с априорной (мы не сообщили модели никаких новых данных по сравнению с теми, на которых основывается априорная вероятность).

In [16]:
# В данном случае, это делать не обязательно, однако, 
# в общем случае, перед каждым запросом следует очистить ранее установленные
# свидетельства
ie.eraseAllEvidence()
# Пусть известно, что у пациента присутствует симптом 1 (про симптом 2 ничего не известно)
ie.setEvidence({'Symptom1': 1})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(gum::Potential<double>@0x23ae59e0980) <Disease:No> :: 0.585799 /<Disease:Yes> :: 0.414201

Наличие информации об имеющемся симптоме существенно увеличивает вероятность того, что у пациента присутствует заболевание. А что если наблюдаются оба симптома?

In [17]:
ie.eraseAllEvidence()
# Пусть известно, что у пациента присутствует симптом 1 (про симптом 2 ничего не известно)
ie.setEvidence({'Symptom1': 1,
                'Symptom2': 1})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(gum::Potential<double>@0x23ae59e0a20) <Disease:No> :: 0.320388 /<Disease:Yes> :: 0.679612

Как и следовало ожидать, вероятность еще выше. Впрочем, она достаточно далека от 1. Видимо, в силу того, что данные симптомы достаточно часто проявляются и без данного заболевания.

# Принятие решения на основе байесовской сети

Простейшим способом принятия решения с испольованием байесовской сети является перебор возможных управлений и оценка ожидаемой полезности для каждого из управлений. А байесовская сеть может использоваться для вычисления вероятностей событий, входящих в оценку полезности. Матрица полезности для данной задачи выглядит следующим образом:

| Болезнь/Disease | Лечить?  |  Эффект  |
|---------|----------|----------|
|  Да     |  Да      |   -1     |
|  Да     |  Нет     |   -40    |
|  Нет    |  Да      |   -10    |
|  Нет    |  Нет     |    0     |

То есть, полезность (эффект) определяется состоянием пациента (болен или нет - переменная Disease) и решением, которое мы принимаем (лечить или нет). Ожидаемая полезность решения определяется как математическое ожидание *Эффекта* при данном решении:

$$
EU(Treat | E) = \sum_{d \in Disease}[p(d|E)*Effect(d, Treat)]
$$

Здесь $E$ - свидетельства, то, что известно в момент принятия решения.

В свою очередь, оптимальным решением является такое, которое максимизирует ожидаемую полезность:

$$
best = \arg\max_{t \in Treat} EU(t | E).
$$

Найдем, например, ожидаемую полезность и оптимальное решение для случая, когда нам ничего неизвестно о симптомах пациента:

In [18]:
import numpy as np

def disease_proba(ie, ev):
    ie.eraseAllEvidence()
    ie.setEvidence(ev)
    return ie.posterior('Disease').toarray()

# Многомерный массив. Измерения:
#   1 - болезнь
#   2 - лечение
utility = np.array([[  0, -10],    # нет болезни
                    [-40,  -1]])   # есть болезнь
#                     ^    ^
#                     |    есть лечение
#                     нет лечения

# Вероятность болезни без данных о симптомах
disease_p = disease_proba(ie, {})

# Ожидаемая стоимость каждого из управлений
eu = np.dot(disease_p, utility) 
print('EU:', eu)

# Управление с наибольшей ожидаемой стоимостью
(np.max(eu), np.argmax(eu))

EU: [-0.4  -9.91]


(-0.4, 0)

То есть, если данных о симптомах нет, то лучше не лечить.

А если есть данные о том, что у пациента присутствует симптом 1:

In [19]:
# Вероятность болезни без данных о симптомах
disease_p = disease_proba(ie, {'Symptom1': 1})

# Ожидаемая стоимость каждого из управлений
eu = np.dot(disease_p, utility) 
print('EU:', eu)

# Управление с наибольшей ожидаемой стоимостью
(np.max(eu), np.argmax(eu))

EU: [-16.56804734  -6.27218935]


(-6.272189349112427, 1)

Таким образом, если наблюдается симптом 1, то оптимальным управлением является лечить пациента.

# Рекомендуемые ссылки:

- https://webia.lip6.fr/~phw//aGrUM/docs/last/notebooks/01-tutorial.ipynb.html и другие обучающие материалы по PyAgrum.
